In [3]:
#pip install transformers torch time pandas
#pip install transformers torch pandas

SyntaxError: invalid syntax (3052838167.py, line 2)

In [ ]:
If you lack access to hardware with a GPU, consider using cloud platforms (e.g., Google Colab).

In [17]:
pip install vaderSentiment

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     |████████████████████████████████| 125 kB 2.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install transformers torch pandas


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import time
import pandas as pd

In [12]:
MODEL_NAME = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

In [13]:
#(WE HAVE TO REPLACE WITH REAL DAATA)
texts = ["The market is experiencing high volatility."] * 1000  # 1000 requests

In [14]:
# Benchmarking testing for the professor!
start_time = time.time()
results = []

for text in texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        sentiment = torch.argmax(outputs.logits).item()  
        results.append(sentiment)

end_time = time.time()

# Calculate statistics
total_time = end_time - start_time
average_time = total_time / len(texts)

# Display results
print(f"Total time for 1000 requests: {total_time:.2f} seconds")
print(f"Average time per request: {average_time:.4f} seconds")

Total time for 1000 requests: 32.66 seconds
Average time per request: 0.0327 seconds


In [15]:
#same but use batch to fine tune 
batch_size = 32
texts = ["The market is volatile."] * 1000  

start_time = time.time()

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    batch_inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**batch_inputs)
        sentiments = torch.argmax(outputs.logits, dim=1) 

end_time = time.time()

total_time = end_time - start_time
average_time = total_time / len(texts)
print(f"Total time: {total_time:.2f} seconds")
print(f"Average time per request: {average_time:.4f} seconds")

Total time: 2.85 seconds
Average time per request: 0.0028 seconds


In [19]:
#if we are allowed to use vader sentiment as we did for ML in Finance
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

start_time = time.time()

results = []
for text in texts:
    sentiment = analyzer.polarity_scores(text)
    results.append(sentiment)

end_time = time.time()
print(f"Total time for 1000 requests: {end_time - start_time:.2f} seconds")

Total time for 1000 requests: 0.01 seconds


# Let's start

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta


In [5]:
def fetch_financial_data(ticker, start_date, end_date, interval="1m"):
    """
    Fetch historical financial data using Yahoo Finance API.
    """
    data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    data.reset_index(inplace=True)
    data.rename(columns={"Datetime": "timestamp"}, inplace=True)  # For consistency
    # Ensure the financial data timestamps are in UTC
    data["timestamp"] = data["timestamp"].dt.tz_convert(None)
    print("Financial Data Sample:")
    print(data.head())
    return data

In [6]:

def generate_synthetic_sentiment_data(start_date, end_date, interval="1min"):
    """
    Generate synthetic sentiment data with timestamps matching the financial data.
    """
    timestamps = pd.date_range(start=start_date, end=end_date, freq=interval, tz=None)
    sentiments = [random.choice(["positive", "neutral", "negative"]) for _ in range(len(timestamps))]
    scores = [random.uniform(-1, 1) for _ in range(len(timestamps))]  # Sentiment intensity
    sentiment_data = pd.DataFrame({
        "timestamp": timestamps,
        "sentiment": sentiments,
        "score": scores
    })
    print("\nSynthetic Sentiment Data Sample:")
    print(sentiment_data.head())
    return sentiment_data

In [7]:
def combine_data(financial_data, sentiment_data):
    """
    Combine financial data with sentiment data based on timestamps.
    """
    financial_data["timestamp"] = pd.to_datetime(financial_data["timestamp"])
    sentiment_data["timestamp"] = pd.to_datetime(sentiment_data["timestamp"])
    
    combined_data = pd.merge_asof(
        sentiment_data.sort_values(by="timestamp"), 
        financial_data.sort_values(by="timestamp"), 
        on="timestamp", 
        direction="backward"
    )
    print("\nCombined Data Sample:")
    print(combined_data.head())
    return combined_data


In [8]:

if __name__ == "__main__":
    # Define parameters
    TICKER = "AAPL"  #^
    START_DATE = (datetime.now() - timedelta(days=5)).strftime("%Y-%m-%d")
    END_DATE = datetime.now().strftime("%Y-%m-%d")
    INTERVAL = "1m" 
    financial_data = fetch_financial_data(TICKER, START_DATE, END_DATE, INTERVAL)
    sentiment_data = generate_synthetic_sentiment_data(START_DATE, END_DATE, interval="1min")
    combined_data = combine_data(financial_data, sentiment_data)
    combined_data.to_csv("combined_financial_sentiment_data.csv", index=False)
    print("\nCombined data saved to 'combined_financial_sentiment_data.csv'.")


[*********************100%***********************]  1 of 1 completed
Financial Data Sample:
            timestamp        Open        High         Low       Close  \
0 2024-12-02 14:30:00  237.270004  238.770004  237.160004  238.700897   
1 2024-12-02 14:31:00  238.690002  239.059906  238.399994  239.000000   
2 2024-12-02 14:32:00  238.960007  239.350006  238.910004  238.934601   
3 2024-12-02 14:33:00  238.929993  239.199997  238.729996  239.110001   
4 2024-12-02 14:34:00  239.080093  239.350006  238.929993  239.296295   

    Adj Close   Volume  
0  238.700897  2562715  
1  239.000000   418941  
2  238.934601   213308  
3  239.110001   192847  
4  239.296295   200970  

Synthetic Sentiment Data Sample:
            timestamp sentiment     score
0 2024-12-01 00:00:00  negative  0.340362
1 2024-12-01 00:01:00   neutral  0.248812
2 2024-12-01 00:02:00   neutral  0.568798
3 2024-12-01 00:03:00  negative -0.079361
4 2024-12-01 00:04:00  negative -0.920842

Combined Data Sample:
          